<a href="https://colab.research.google.com/github/racoope70/Machine-Learning-Based-Sentiment-Analysis-of-IMDb-Movie-Reviews/blob/main/IMDB_Sentiment_Analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %%
# Import necessary libraries
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.model_selection import train_test_split
from google.colab import drive
import tensorflow as tf
from sklearn.metrics import accuracy_score
import gc  # Import garbage collection for memory management
from huggingface_hub import notebook_login


In [2]:
# %%
# Step 1: Install required libraries (run this only once to install dependencies)
!pip install transformers pandas scikit-learn torch torchvision torchaudio matplotlib seaborn


In [3]:
# Step 2: Load IMDb Dataset
def load_imdb_dataset(base_path):
    """
    Loads the IMDb dataset from given directory structure and returns a DataFrame with reviews and sentiments.
    """
    reviews = []
    sentiments = []

    # Iterate over both train and test data
    for split in ['train', 'test']:
        for sentiment in ['pos', 'neg']:
            path = os.path.join(base_path, split, sentiment)
            # Read reviews from each file
            for file_name in os.listdir(path):
                with open(os.path.join(path, file_name), 'r', encoding='utf-8') as file:
                    reviews.append(file.read())
                    sentiments.append(1 if sentiment == 'pos' else 0)

    return pd.DataFrame({'review': reviews, 'sentiment': sentiments})

In [4]:
# Step 3: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Step 4: Define paths for saving datasets and results
base_path = '/content/drive/MyDrive/aclImdb'
csv_file_path = '/content/drive/MyDrive/aclImdb_reviews.csv'
results_dir = '/content/drive/MyDrive/sentiment_analysis_results'  # Define the directory for saving results

# Ensure the results directory exists
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

if os.path.exists(csv_file_path):
    df = pd.read_csv(csv_file_path)
else:
    df = load_imdb_dataset(base_path)
    df.to_csv(csv_file_path, index=False)


In [6]:
# Step 5: Sample a subset of the data
data = df.sample(10000)


In [7]:
# Step 6: Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['review'], data['sentiment'], test_size=0.2, random_state=42
)

# Free up memory after splitting
del df, data
gc.collect()

60

In [8]:
# Step 7: Authenticate with Hugging Face Hub (Optional)
# This allows for better access and avoids rate limits when downloading public models/datasets
#notebook_login()


In [9]:
# Step 8: Tokenize Text using BERT Tokenizer in Batches
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(texts, tokenizer, batch_size=64):
    """
    Tokenizes input texts in smaller batches to optimize memory usage.
    """
    tokenized_data = {"input_ids": [], "attention_mask": []}
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size].tolist()
        encodings = tokenizer(batch, truncation=True, padding=True, max_length=256)
        tokenized_data["input_ids"].extend(encodings["input_ids"])
        tokenized_data["attention_mask"].extend(encodings["attention_mask"])
    return tokenized_data

# Tokenize training and testing data
train_encodings = tokenize_data(train_texts, tokenizer)
test_encodings = tokenize_data(test_texts, tokenizer)

# Free memory after tokenization
del train_texts, test_texts
gc.collect()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

0

In [10]:
# %%
# Step 9: Define Dataset Class
class SentimentDataset(torch.utils.data.Dataset):
    """
    Custom Dataset class for sentiment analysis.
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Prepare training and testing datasets
train_dataset = SentimentDataset(train_encodings, list(train_labels))
test_dataset = SentimentDataset(test_encodings, list(test_labels))

# Free memory after creating datasets
del train_encodings, test_encodings, train_labels, test_labels
gc.collect()

0

In [11]:
# %%
# Step 10: Set up the model with BERT and GPU support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Step 11: Define Training Arguments with Checkpoints
training_args = TrainingArguments(
    output_dir=results_dir,  # Save checkpoints to Google Drive
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,  # Limit the number of saved checkpoints
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.1,
    logging_dir=os.path.join(results_dir, "logs"),  # Log directory
    fp16=True,  # Mixed precision training
    load_best_model_at_end=True,  # Load the best model
    report_to="tensorboard",
)

In [13]:
# %%
# Step 12: Train Model with Checkpoints
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Free memory before training
gc.collect()


61

In [14]:
# %%
# Step 12: Start the training process
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.279562
2,No log,0.265898
3,No log,0.300169


TrainOutput(global_step=375, training_loss=0.9448507486979166, metrics={'train_runtime': 330.9703, 'train_samples_per_second': 72.514, 'train_steps_per_second': 1.133, 'total_flos': 3157332664320000.0, 'train_loss': 0.9448507486979166, 'epoch': 3.0})

In [15]:
# %%
# Step 13: Evaluate the model's performance
# Ensure that test labels and predictions are on the same device
preds = trainer.predict(test_dataset).predictions


# Calculate accuracy
test_labels_tensor = torch.tensor(list(test_dataset.labels)).to(torch.device("cpu"))
binary_preds = torch.argmax(torch.tensor(preds), dim=1)
accuracy = accuracy_score(test_labels_tensor.numpy(), binary_preds.numpy())

print(f"Accuracy: {accuracy}")

Accuracy: 0.8945
